# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.project.domain.events.ProjectEvents.ProjectEvents import ProjectCreatedEvent


class Invalid:
    pass


class ProjectorInvalid0:
    @listen_to(ProjectCreatedEvent)
    def on_project_created(self):
        print('Created project')


class ProjectorInvalid1:
    @listen_to(ProjectCreatedEvent)
    def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
        print('Created project')


class ProjectorInvalid2:
    @listen_to(ProjectCreatedEvent)
    def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
        print('Created project')


class ProjectorInvalid3:
    @listen_to(ProjectCreatedEvent)
    def on_project_created(self, event: Invalid, metadata: EventMetadata):
        print('Created project')


class ProjectorInvalid4:
    @listen_to(ProjectCreatedEvent)
    def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
        print('Created project')


event_publisher = EventPublisher()
try:
    projector = ProjectorInvalid0()
    event_publisher.register(projector)
except Exception as e:
    print(e)

try:
    projector = ProjectorInvalid1()
    event_publisher.register(projector)
except Exception as e:
    print(e)

try:
    projector = ProjectorInvalid2()
    event_publisher.register(projector)
except Exception as e:
    print(e)

try:
    projector = ProjectorInvalid3()
    event_publisher.register(projector)
except Exception as e:
    print(e)

try:
    projector = ProjectorInvalid4()
    event_publisher.register(projector)
except Exception as e:
    print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x1373a2540>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x138e07fe0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x1373a2540>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x138e07fe0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x1373a2540>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
    @listen_to(ProjectCreatedEvent)
    def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
        print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
    event_publisher = EventPublisher()
    projector = Projector()
    event_publisher.register(projector)
    event_publisher.register(projector)
except Exception as e:
    print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.common.types.identity.Identity import UserId
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.types.Project import Description, Name, ProjectId, Tags

for i in range(10):
    create_project_command = CreateProjectCommand(
        project_id=ProjectId.new(),
        name=Name.from_str(f'Project {i}'),
        description=Description.from_str(f'Description {i}'),
        tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
        user_id=UserId.new(),
    )
    CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
    print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
    print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 2038 events
"Project Created" occurred at 2026-02-19T19:06:23.542799-03:00
"Project Created" occurred at 2026-02-19T19:06:23.549815-03:00
"Project Created" occurred at 2026-02-19T19:06:23.552344-03:00
"Project Created" occurred at 2026-02-19T19:06:23.554923-03:00
"Project Created" occurred at 2026-02-19T19:06:23.556756-03:00
"Project Created" occurred at 2026-02-19T19:06:23.558591-03:00
"Project Created" occurred at 2026-02-19T19:06:23.560313-03:00
"Project Created" occurred at 2026-02-19T19:06:23.561972-03:00
"Project Created" occurred at 2026-02-19T19:06:23.563688-03:00
"Project Created" occurred at 2026-02-19T19:06:23.565388-03:00
"Project Created" occurred at 2026-02-19T19:11:29.173970-03:00
"Model Created" occurred at 2026-02-19T19:11:29.355037-03:00
"Version Created" occurred at 2026-02-19T19:11:29.365934-03:00
"Version Assigned to Model" occurred at 2026-02-19T19:11:29.367487-03:00
"Calculation Profile Added" occurred at 2026-02-19T19:11:29.371249-03:00
"Project Calculatio

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries


------------------------------


Found 2038 events


Reset projection


Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
    print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 363 project summaries
Project ab7f846e-8d35-419f-b964-7c53c1894d41: Project 0
Project 2cbc6db5-6440-4bc3-acdc-31f749d80c4b: Project 1
Project d170b2c8-5c65-4c10-805d-f1387a04a694: Project 2
Project 96d522cb-6009-436f-adc4-12e679da0c27: Project 3
Project ec0ec932-01bb-45fd-912c-84e53bebaa6c: Project 4
Project acd7c05a-a03d-4d25-be69-86826b0657a4: Project 5
Project d5a10534-6b59-47a6-bdab-a3147a139371: Project 6
Project e9936ea8-7009-4a5b-aaba-cf561525c2f3: Project 7
Project 499e7b10-0b95-4fea-b7a0-cf106baa67fc: Project 8
Project 51663885-1f9b-4819-b100-ba88124f178a: Project 9
Project c0a15f70-5dd4-4b81-99a1-b8c721bb10dc: Rio Primero 123
Project 9d942fb1-6005-4f4c-913e-52e48a7fa447: Project 0
Project f462d8d5-b7f3-437f-91f2-dffd4bf0be69: Project 1
Project 1a5fc67a-1711-4c28-8c29-484c46e9009d: Project 2
Project 6fcf0054-4d58-4e53-8a50-1c36d0c66857: Project 3
Project 6760fe45-6abc-42c3-a150-401217a5980e: Project 4
Project 6e99db62-7a4a-4bd2-b2a5-2a05e51b0b50: Project 5
Project 913391